<a href="https://colab.research.google.com/github/KrlvIvn/Praktikum/blob/main/12_NLP_project_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

**Комментарий студента:**
- проект выполнен на Google Colab;  
- ориентировочное время выполнение 20 минут.

## Подготовка
Работа выполнена в Google Colab (TPU).  
Установка библиотеки transformers


In [ ]:
!pip install transformers

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


Импорт необходимых библиотек:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score,make_scorer
 
import torch
import transformers as ppb
from tqdm import notebook
 
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.dummy import DummyClassifier
 
 
nltk.download('wordnet','stopwords','punkt','averaged_perceptron_tagger')
from nltk.corpus import wordnet
 
 
import warnings
warnings.filterwarnings('ignore')

## Импорт данных


In [ ]:
try:
    comments = pd.read_csv('/datasets/toxic_comments.csv')
except:
    comments = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [ ]:
comments.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


Преимущественно не токсичные комментарии соотношение 89,8% к 10,2% 

In [ ]:
comments.toxic.value_counts()/comments.shape[0]*100 #.plot(kind='bar')

0    89.832112
1    10.167888
Name: toxic, dtype: float64

In [ ]:
def df_info(df):
    '''
  функция дает общее представление о датасете
  
  '''  
    #print(80*"+")
    df.info()
    print(80*"+")
    print(f'количество пропусков по столбцам:')
    for i,j in enumerate(df.columns):
      print(j,'=',df.iloc[:,i].isnull().sum())
    print('Описание количественных переменных:')
    print(df.describe(percentiles=[.25,.5]))
    print(80*"+")
    print(f'количество дубликатов в таблице = {df.duplicated().sum()}')

In [ ]:
df_info(comments)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
количество пропусков по столбцам:
text = 0
toxic = 0
Описание количественных переменных:
               toxic
count  159571.000000
mean        0.101679
std         0.302226
min         0.000000
25%         0.000000
50%         0.000000
max         1.000000
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
количество дубликатов в таблице = 0


In [ ]:
comments_new = []
for sentence in comments.text:
 
   comments_new.append(" ". join(sentence.split()))

In [ ]:
comments["text_split"]=comments_new

## Обучение

Для дальнейшего использования 159571 очень большой датасет, поэтому сделаем sample.


переменную **corpus** будем использовать для TF-IDF,  
**corpus_b** для модели BERT

In [ ]:
sample_size = 30000
sample_size_b = 5000

In [ ]:
corpus = comments.sample(n=sample_size,random_state=0).reset_index()
corpus_b = comments.sample(n=sample_size_b,random_state=0).reset_index()
 
corpus = corpus.drop('index',axis=1)
corpus_b = corpus_b.drop('index',axis=1)

In [ ]:
 
print('соотношение классов в датасете corpus\n', corpus.toxic.value_counts()/corpus.shape[0]*100)
 
print('соотношение классов в датасете corpus_b\n',corpus_b.toxic.value_counts()/corpus_b.shape[0]*100)

соотношение классов в датасете corpus
 0    89.646667
1    10.353333
Name: toxic, dtype: float64
соотношение классов в датасете corpus_b
 0    90.1
1     9.9
Name: toxic, dtype: float64


Соотношение классов приблизтельно как в исходном датасете.  
Можно считать, что baseline по accuracy для данного датасета около  89,7 - 90.1.

# TF-IDF модель

Будем применять Wordnet Lemmatizer

In [ ]:
corpus.head()

,text,toxic,text_split
0,"""\nI haven't paraphrased you at all, Gary. Yo...",0,""" I haven't paraphrased you at all, Gary. You ..."
1,I BLOCKED REVERS! I BLOCKED REVERS! I BLOCKED ...,1,I BLOCKED REVERS! I BLOCKED REVERS! I BLOCKED ...
2,I'm sorry. I'd like to unreservedly retract my...,1,I'm sorry. I'd like to unreservedly retract my...
3,I don't know if this is exactly like the Press...,0,I don't know if this is exactly like the Press...
4,"Thank you all, we'll all improve the Wikipedia...",0,"Thank you all, we'll all improve the Wikipedia..."


In [ ]:
# инициализация Wordnet Lemmatizer
L = WordNetLemmatizer()

In [ ]:
def lemmatizered(corpus):
  ''' функция выполняет токенизациию и лемматизацию массива текстов'''
  corpus_new = []
  for sentence in corpus:
    word_list = nltk.word_tokenize(sentence)
    corpus_new.append(' '.join([L.lemmatize(w) for w in word_list]))
  return corpus_new

In [ ]:
def get_wordnet_pos(word):
    """функция возвращает словарь, где возвращается значение часть речи (pos_tag)"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
 
    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
def get_word_text(corpus):
  ''' функция выполняет токенизациию и лемматизацию массива текстов c учетом pos_tag'''
  corpus_new = []
  for sentence in corpus:
    corpus_new.append(' '.join([L.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)]))
  return corpus_new

приведем все слова к нижнему регистру

In [ ]:
corpus['text_split'] = corpus['text_split'].str.lower()
corpus.head()

,text,toxic,text_split
0,"""\nI haven't paraphrased you at all, Gary. Yo...",0,""" i haven't paraphrased you at all, gary. you ..."
1,I BLOCKED REVERS! I BLOCKED REVERS! I BLOCKED ...,1,i blocked revers! i blocked revers! i blocked ...
2,I'm sorry. I'd like to unreservedly retract my...,1,i'm sorry. i'd like to unreservedly retract my...
3,I don't know if this is exactly like the Press...,0,i don't know if this is exactly like the press...
4,"Thank you all, we'll all improve the Wikipedia...",0,"thank you all, we'll all improve the wikipedia..."


Выполним лемматизацию корпуса c учетом pos_tag

In [ ]:
 
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
 
corpus['lemma_text'] = get_word_text(corpus['text_split'])
corpus.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,text,toxic,text_split,lemma_text
0,"""\nI haven't paraphrased you at all, Gary. Yo...",0,""" i haven't paraphrased you at all, gary. you ...","`` i have n't paraphrase you at all , gary . y..."
1,I BLOCKED REVERS! I BLOCKED REVERS! I BLOCKED ...,1,i blocked revers! i blocked revers! i blocked ...,i block revers ! i block revers ! i block reve...
2,I'm sorry. I'd like to unreservedly retract my...,1,i'm sorry. i'd like to unreservedly retract my...,i 'm sorry . i 'd like to unreservedly retract...
3,I don't know if this is exactly like the Press...,0,i don't know if this is exactly like the press...,i do n't know if this be exactly like the pres...
4,"Thank you all, we'll all improve the Wikipedia...",0,"thank you all, we'll all improve the wikipedia...","thank you all , we 'll all improve the wikiped..."


Создадим переменную stopwords и передадим в нее список стоп слов английского языка из библ.nltk

In [ ]:
 
 
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


разделим наши данные на train_corpus ,test_corpus.  
test_corpus в дальнейшем будем использовать для тестирования модели.

In [ ]:
train_corpus,test_corpus = train_test_split(corpus, test_size=0.5)

выполним векторизацию слов методом TfidfVectorizer, при этом удалим из текста стоп-слова.   
Метод fit настривает модель, transform выполняет вектризацию.

In [ ]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit(train_corpus['lemma_text'])
tf_idf_train = tf_idf.transform(train_corpus['lemma_text'])

In [ ]:
test_tf_idf = tf_idf.transform(test_corpus['lemma_text'])

In [ ]:
print("Размер матрицы train:", tf_idf_train.shape)
print("Размер матрицы test:", test_tf_idf.shape)

Размер матрицы train: (15000, 40216)
Размер матрицы test: (15000, 40216)


In [ ]:
tf_idf_train.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

Обозначим что есть признаки и таргеты для трейн и тест.

In [ ]:
X_train = tf_idf_train
X_test = test_tf_idf
y_train = train_corpus['toxic'].values
y_test = test_corpus['toxic'].values

In [ ]:
print(X_train.shape,X_test.shape)

(15000, 40216) (15000, 40216)


Создадим скоринг для кросс-валидации.  
По заданию нужно добиться значения F-меры более 0.75

In [ ]:
f1 = make_scorer(f1_score , average='macro')

In [ ]:
def treshold(array,tresh_list,y_test):
  ''' функция настраивает порог классификации для метода predict_proba'''
  best_result = 0
  #best_predict = []
  for i in tresh_list:
    tmp_list = np.empty([len(array),2])
    for j,k in enumerate(array[:,1]):
      if k<i:
        tmp_list[j,1] = 0
      else:
        tmp_list[j,1] = 1
    result = f1_score(y_test,tmp_list[:,1])
    #print(result,i)   
    if result>best_result:
      best_result=result
      best_tresh = i
      #best_predict = tmp_list
  return best_result,best_tresh

In [ ]:
result_df = pd.DataFrame(columns=['F1_predict','F1_predict_proba','Treshold'])
result_df

,F1_predict,F1_predict_proba,Treshold


### Логистическая классификация для TF-IDF

In [ ]:
 parameters = {'C': np.linspace(0.0001, 5000, 50),'max_iter':[100,200],'class_weight':['balanced', None]}
 grid_search = GridSearchCV(LogisticRegression(random_state=0), parameters,scoring=f1,n_jobs=-1,cv=5)
 grid_search.fit(X_train, y_train)
 
 print('best parameters: ', grid_search.best_params_)
 print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 102.04091428571428, 'class_weight': 'balanced', 'max_iter': 100}
best scrores:  0.8492330207759704


In [ ]:
lr_clf_TF_IDF = LogisticRegression(**grid_search.best_params_,random_state = 0)
lr_clf_TF_IDF.fit(X_train, y_train)

LogisticRegression(C=102.04091428571428, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
f1_lr_clf_train = f1_score(lr_clf_TF_IDF.predict(X_train),y_train)
f1_lr_clf_test = f1_score(lr_clf_TF_IDF.predict(X_test),y_test)
print('f1_score для TF_IDF на тренировочной выборке =', f1_lr_clf_train)
print('f1_score для TF_IDF на тестовой выборке =',f1_lr_clf_test)

f1_score для TF_IDF на тренировочной выборке = 0.9974009096816114
f1_score для TF_IDF на тестовой выборке = 0.7331975560081466


подберем порог классификации

In [ ]:
TF_IDF_predict_proba = lr_clf_TF_IDF.predict_proba(X_test)

In [ ]:
lr_clf_TF_IDF_result,lr_clf_TF_IDF_tresh = treshold(TF_IDF_predict_proba,np.linspace(0.1,0.9,50),y_test)
print('лучшее значение f1 и порог равны',lr_clf_TF_IDF_result,lr_clf_TF_IDF_tresh)

лучшее значение f1 и порог равны 0.7331738981892724 0.5244897959183674


In [ ]:
def result_write(model_name,f1,f1_p,t):
  result_df.loc[model_name,'F1_predict'] = f1
  result_df.loc[model_name,'F1_predict_proba'] = f1_p
  result_df.loc[model_name,'Treshold'] = t
  return result_df

In [ ]:
result_write('Log reg TF-IDF',f1_lr_clf_test,lr_clf_TF_IDF_result,lr_clf_TF_IDF_tresh)

,F1_predict,F1_predict_proba,Treshold
Log reg TF-IDF,0.733198,0.733174,0.52449


## Random Forest для TF-IDF

In [ ]:
parameters = {'n_estimators': [500,1000]} #'max_depth' : [280,300,320]
grid_rfc = GridSearchCV(RandomForestClassifier(random_state=0), parameters,scoring=f1,n_jobs=-1,cv=3)
grid_rfc.fit(X_train, y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=0,
                                   

In [ ]:
print('best parameters: ', grid_rfc.best_params_)
print('best scrores: ', grid_rfc.best_score_)

best parameters:  {'n_estimators': 500}
best scrores:  0.813438432225964


In [ ]:
rfc = RandomForestClassifier(**grid_rfc.best_params_)
rfc.fit(X_train, y_train)





RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
f1_rfc_train = f1_score(rfc.predict(X_train),y_train)
f1_rfc_test = f1_score(rfc.predict(X_test),y_test)
print('f1_score для TF_IDF на тренировочной выборке =', f1_rfc_train)
print('f1_score для TF_IDF на тестовой выборке =',f1_rfc_test)






f1_score для TF_IDF на тренировочной выборке = 0.9993489583333334
f1_score для TF_IDF на тестовой выборке = 0.6803874092009685


In [ ]:
rfc_predict_proba = rfc.predict_proba(X_test)
rfc_TF_IDF_result,rfc_TF_IDF_tresh = treshold(rfc_predict_proba,np.linspace(0.1,0.9,50),y_test)
print('лучшее значение f1 и порог равны',rfc_TF_IDF_result,rfc_TF_IDF_tresh)

лучшее значение f1 и порог равны 0.7605829285218599 0.3122448979591837


In [ ]:
result_write('Random_forest TF-IDF',f1_rfc_test,rfc_TF_IDF_result,rfc_TF_IDF_tresh)

,F1_predict,F1_predict_proba,Treshold
Log reg TF-IDF,0.733198,0.733174,0.52449
Random_forest TF-IDF,0.680387,0.760583,0.312245


Получили необходимое значение F1 для RandomForest

## DummyClassifier

In [ ]:

clf = DummyClassifier(random_state=0)
clf.fit(X_train, y_train)
Dummy_predict = clf.predict(X_test)
Dummy_predict_proba = clf.predict_proba(X_test)
dummy_TF_IDF_result,dummy_TF_IDF_tresh = treshold(Dummy_predict_proba,np.linspace(0.001,0.9,50),y_test)

print("Dummy classifier score: " ,f1_score(y_test,Dummy_predict))

Dummy classifier score:  0.10894941634241244


In [ ]:
result_write('DummyClassifier',f1_score(y_test,Dummy_predict),dummy_TF_IDF_result,dummy_TF_IDF_tresh)

,F1_predict,F1_predict_proba,Treshold
Log reg TF-IDF,0.733198,0.733174,0.52449
Random_forest TF-IDF,0.680387,0.760583,0.312245
DummyClassifier,0.108949,0.108949,0.001


#BERT

Для дальнейшей работы изменим размер(длину) текстов до 512 символов.. Это необходимо, чтобы корректно(без ошибок) выполнить токенизацию. И далее когда будем использовать BERT модель , то получим ошибку по максимальной длине токенов = 512.  
 
Описание предтренированных данных сказано:
12-layer, 768-hidden, 12-heads, 110M parameters.
Trained on **uncased English**, т.е. к нижнему регистру приводить не надо.

In [ ]:
for k,i in enumerate(corpus_b['text_split']):
  longest = 0
  if len(corpus_b['text_split'][k])>longest:
    longest = len(corpus_b['text_split'][k])
print(longest)

351


In [ ]:
corpus_b['text_512'] = np.zeros(corpus_b.shape[0])

In [ ]:
for k,i in enumerate(corpus_b['text_split']):
    corpus_b['text_512'][k]=i[:512]
    #comments['text_512'][k]=i[-512:] Попробовал взять последние 512 слов, результат тот же.

In [ ]:
corpus_b.head()

,text,toxic,text_split,text_512
0,"""\nI haven't paraphrased you at all, Gary. Yo...",0,""" I haven't paraphrased you at all, Gary. You ...",""" I haven't paraphrased you at all, Gary. You ..."
1,I BLOCKED REVERS! I BLOCKED REVERS! I BLOCKED ...,1,I BLOCKED REVERS! I BLOCKED REVERS! I BLOCKED ...,I BLOCKED REVERS! I BLOCKED REVERS! I BLOCKED ...
2,I'm sorry. I'd like to unreservedly retract my...,1,I'm sorry. I'd like to unreservedly retract my...,I'm sorry. I'd like to unreservedly retract my...
3,I don't know if this is exactly like the Press...,0,I don't know if this is exactly like the Press...,I don't know if this is exactly like the Press...
4,"Thank you all, we'll all improve the Wikipedia...",0,"Thank you all, we'll all improve the Wikipedia...","Thank you all, we'll all improve the Wikipedia..."


## Загружаем предтренированную модель BERT


Перебпровал различные предтренированные модели.  
Лучшие показатели у 'bert-base-uncased'.  
Модель принимает значение в обоих регистрах!  
Обучена на BookCorpus, 11 038 неопубликлванных книг и English Wikipedia (исключая списки, таблицы,заголовки).

In [ ]:
configuration = ppb.BertConfig(max_position_embeddings=2048)

In [ ]:
## BERT:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel(configuration), ppb.BertTokenizer, 'bert-base-uncased') 
# results:
# bert-base-uncased 0.66 
# bert-large-uncased 0.59
# bert-base-uncased-finetuned-mrpc 0.66
# bert-base-multilingual-uncased 0.45
# bert-large-cased-whole-word-masking-finetuned-squad 0.59
 
 
# Загружаем предтренированную model/tokenizer BERT
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model_tokenizer_class = model_class.from_pretrained(pretrained_weights)

 
### Токенизация.
 
Токенизируем каждый текст , т.е. приводим в формат для BERT моделей.

In [ ]:
tokenized= corpus_b['text_512'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

Принцип токенизации.  
Взято с блока Jay Alammar.
<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tokenization-2-token-ids.png" />

### Padding (заполнение)
После токенизации, каждый токенизированный текст имеет разную длину.
Для дальнейшей работы BERT модели необходимо привести все "тексты" к одной размерности.  
Для этого добавим 0 до размера самого большого текста. 


In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
 
print(max_len)

357


In [ ]:
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
print(padded.shape)

(5000, 357)


Размерность датасета 5000 строк на 357 столбцов.  
Все строки одной длины.

### Masking (маскирование)  
Если подать данные в таком виде в BERT модель, то модель не сработает, дело в добавленных 0. Необходимо создать еще одную переменную, чтобы она маскировала добавленные 0(padding). Поэтому создадим  attention_mask:

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
print(attention_mask.shape)

(5000, 357)


In [ ]:
batch_size = 1000
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.IntTensor(padded[batch_size*i:batch_size*(i+1)])
        attention_mask_batch = torch.IntTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        with torch.no_grad():
            batch_embeddings = model_tokenizer_class(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())
 
embeddings[:1]

[array([[ 0.17508629,  0.06400632, -0.19117032, ..., -0.1570864 ,
          0.49064207,  0.41450417],
        [-0.29137355, -0.10913293,  0.20857978, ...,  0.02302492,
          0.2964966 ,  0.68998164],
        [ 0.01597106, -0.15524422, -0.37359148, ..., -0.02488263,
          0.17746924,  0.57595164],
        ...,
        [-0.07746883, -0.31962118, -0.0884318 , ..., -0.54371226,
          0.5737397 ,  0.7110472 ],
        [ 0.17904532,  0.07629653,  0.09958477, ..., -0.02940109,
          0.41762403,  0.40146947],
        [-0.0606179 ,  0.05414834, -0.00974915, ..., -0.04243546,
          0.20950316,  0.50788856]], dtype=float32)]

Созадим признаки для обучения моделей классификации

In [ ]:
features = np.concatenate(embeddings)
print(features.shape)

(5000, 768)


In [ ]:
labels = corpus_b['toxic']
print(labels.shape)

(5000,)


## Логистическая классификация для BERT 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels,test_size=0.5)

Выполним подбор коэфф.регуляризации:

In [ ]:
 parameters = {'C': np.linspace(0.0001, 5000, 50),'class_weight':['balanced']}
 grid_search_bert = GridSearchCV(LogisticRegression(random_state=0), parameters,scoring=f1,n_jobs=-1,cv=5)
 grid_search_bert.fit(X_train, y_train)
 
 print('best parameters: ', grid_search_bert.best_params_)
 print('best scores: ', grid_search_bert.best_score_)

best parameters:  {'C': 918.3674285714285, 'class_weight': 'balanced'}
best scores:  0.7984095314425541


In [ ]:
lr_clf_BERT = LogisticRegression(**grid_search_bert.best_params_,random_state=0)
lr_clf_BERT.fit(X_train, y_train)

LogisticRegression(C=918.3674285714285, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
BERT_predict_train = lr_clf_BERT.predict(X_train)
BERT_predict = lr_clf_BERT.predict(X_test)
BERT_predict_proba = lr_clf_BERT.predict_proba(X_test)

In [ ]:
f1_lr_clf_train_bert = f1_score(BERT_predict_train,y_train)
f1_lr_clf_test_bert = f1_score(BERT_predict,y_test)
print('f1_score для BERT на тренировочной выборке =', f1_lr_clf_train_bert)
print('f1_score для BERT на тестовой выборке =',f1_lr_clf_test_bert)

f1_score для BERT на тренировочной выборке = 1.0
f1_score для BERT на тестовой выборке = 0.5646123260437375


In [ ]:
lr_clf_BERT_result,lr_clf_BERT_tresh = treshold(BERT_predict_proba,np.linspace(0.1,0.9,50),y_test)
print('лучшее значение f1 и порог равны',lr_clf_BERT_result,lr_clf_BERT_tresh)

лучшее значение f1 и порог равны 0.5767634854771786 0.753061224489796


In [ ]:
result_write('Log reg BERT',f1_lr_clf_test_bert,lr_clf_BERT_result,lr_clf_BERT_tresh)

,F1_predict,F1_predict_proba,Treshold
Log reg TF-IDF,0.733198,0.733174,0.52449
Random_forest TF-IDF,0.680387,0.760583,0.312245
DummyClassifier,0.108949,0.108949,0.001
Log reg BERT,0.564612,0.576763,0.753061


## Random Forest для BERT

 Подберем гиперпараметры для случайного леса.

In [ ]:
parameters = {'n_estimators': [80,100],'max_depth' : [3,5,10], 'class_weight' : ['balanced']}
grid_rfc_BERT = GridSearchCV(RandomForestClassifier(random_state=0), parameters,scoring=f1,n_jobs=-1,cv=5)
grid_rfc_BERT.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=0,
                                   

In [ ]:
print('best parameters: ', grid_rfc_BERT.best_params_)
print('best scrores: ', grid_rfc_BERT.best_score_)

best parameters:  {'class_weight': 'balanced', 'max_depth': 5, 'n_estimators': 80}
best scrores:  0.7970155790010286


In [ ]:
rfc_Bert = RandomForestClassifier(**grid_rfc_BERT.best_params_)
rfc_Bert.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=80,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
f1_rfc_Bert_train = f1_score(rfc_Bert.predict(X_train),y_train)
f1_rfc_Bert_test = f1_score(rfc_Bert.predict(X_test),y_test)
print('f1_score для BERT на тренировочной выборке =', f1_rfc_Bert_train)
print('f1_score для BERT на тестовой выборке =',f1_rfc_Bert_test)

f1_score для BERT на тренировочной выборке = 0.897887323943662
f1_score для BERT на тестовой выборке = 0.5972222222222221


In [ ]:
rfc_BERT_predict_proba = rfc_Bert.predict_proba(X_test)
rfc_BERT_result,rfc_BERT_tresh = treshold(rfc_BERT_predict_proba,np.linspace(0.1,0.9,50),y_test)
print('лучшее значение f1 и порог равны',rfc_BERT_result,rfc_BERT_tresh)

лучшее значение f1 и порог равны 0.6096491228070176 0.4755102040816327


In [ ]:
result_write('Random_forest BERT',f1_rfc_Bert_test,rfc_BERT_result,rfc_BERT_tresh)

,F1_predict,F1_predict_proba,Treshold
Log reg TF-IDF,0.733198,0.733174,0.52449
Random_forest TF-IDF,0.680387,0.760583,0.312245
DummyClassifier,0.108949,0.108949,0.001
Log reg BERT,0.564612,0.576763,0.753061
Random_forest BERT,0.597222,0.609649,0.47551


# Вывод:
1. выполнено создание  признаков для текстов моделью BERT и TF_IDF;
2. Далее с помощью логистической регрессии и случ.леса выполнены настройки моделей, с подбором гиперпараметров;
3. Единственная модель которая выдала необходимый результат это TF_IDF + Random_Forest;
4. Для построения признаков TF-IDF использовал 30000 строк, для BERT - 5000 строк;
5. Если увеличить выборку , то результаты улучшатся незначительно + это занимает очень много времени;
5. Т.к. при постоении признаков для BERT моделли есть ограничение в 512 слов, то пришлось сделать ограничение. Считаю , из-за этого не удалось показать хорошие результаты для BERT.
